In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
import datetime as dt
import matplotlib.pyplot as plt
import structure_product

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [70]:
import importlib
importlib.reload(structure_product)

<module 'structure_product' from 'd:\\QF5202\\Projects\\Project2_SFD500\\structure_product.py'>

### Load data and set up basic parameters

In [1]:
stock_df = pd.read_csv("Data\Project2 SFD500-stock_price-002727.csv").rename(columns = {"Unnamed: 0": "date"}) # dataframe of the underlying stock price
stock_series = stock_df.set_index(pd.to_datetime(stock_df["date"]))["CLOSE"].squeeze() # time series of stock price
logret_series = np.log(stock_series).diff().dropna() # time series of log return with the first day deleted
dividend_df = pd.read_csv("Data\dividends_002027.csv") # dataframe of annual cash dividends of the underlying stock
dividend_series = dividend_df.set_index(pd.to_datetime(dividend_df["Date"]))["Dividends"].squeeze() # time series of annual cash dividend
trade_date = pd.to_datetime(pd.read_csv("Data\Project2 business_day.csv")["date"]) # trading dates from 2019-01-03 to 2020-01-06

NameError: name 'pd' is not defined

In [27]:
trade_dates = pd.to_datetime(pd.read_csv('Data\Project2 business_day.csv')['date'])
## assert constant
startDate, endDate, valueDate = dt.datetime(2019,1,4), dt.datetime(2019,12,27), dt.datetime(2019,1,3)
# monitor dates for up-and-out barrier
UnO_dates = pd.to_datetime(pd.Series([dt.date(2019,2,1)
             ,dt.date(2019,3,4)
             ,dt.date(2019,3,29)
             ,dt.date(2019,4,26)
             ,dt.date(2019,5,31)
             ,dt.date(2019,6,28)
             ,dt.date(2019,7,26)
             ,dt.date(2019,8,23)
             ,dt.date(2019,9,20)
             ,dt.date(2019,10,25)
             ,dt.date(2019,11,22)
             ,dt.date(2019,12,27)]))
face_value = 100
r = np.log(1 + 0.015) 
# r = np.log(1 + 0.0269) # continuously compounded risk-free rate
sigma = np.sqrt(252*(logret_series**2).mean())
# sigma = stock_series.pct_change().std() * np.sqrt(252)
# T = (endDate - valueDate).days / 365 # time to maturity in year (358/365)
# sigma = pricing_mc.get_vol_option_garch11(T, logret_series, sample_size = 90) # estimated volatility of the stock over the period
q = structure_product.get_dividend_yield(stock_series, dividend_series) # continuously compounded dividend yield
n = 100000 # number of sample paths
S0 = 5.10 # initial stock price on 2019-01-03
R = 0.28 # annualized coupon rate
UnO_factor = 1.05
DnI_factor = 0.68
DnI_dates = trade_dates[trade_dates <= endDate]
strike = 1 # strike price
print(f"r: {r}\nq: {q}\nsigma: {sigma}")

r: 0.014888612493750559
q: 0.0127009829447111
sigma: 0.507611640063509


### Initiate the structure product and estimate price on the valueation date

In [71]:
SFD500 = structure_product.SFD500(startDate, endDate, face_value, S0, UnO_factor, DnI_factor, UnO_dates, DnI_dates, R, strike, trade_dates)

In [28]:
v0, se, time, v0_df = SFD500.get_mc_price(valueDate, S0, r, q, sigma, n, knock_in = False) 
print(f"{v0}\n{se}")

96.25741907114343
0.06882014743426405


### Compare Monte-Carlo estimates of returns with the real return

In [8]:
v_df = SFD500.get_mc_price_daily(r, q, sigma, 100, v0) # daily MC estimates of price 
v_df.to_csv("mc_return.csv")
mc_daily_mean = v_df.mean(axis = 0) 
mc_daily_median = v_df.median(axis = 0)
mc_daily_std = v_df.std(axis = 0)

In [30]:
S_real = pd.read_csv("Data\price_002027_190103_191227.csv").rename(columns = {"Date": "date"}) # underlying stock price during the product period
S_real["date"] = pd.to_datetime(S_real["date"])
S_real = S_real.merge(trade_dates[trade_dates <= endDate], how = "right", on = "date").set_index("date")["Close"].ffill() # fill in missing stock price with the previous price
v_real = SFD500.get_real_price_daily(S_real, r, q, sigma, n, v0) # daily MC estimates of price using market data of stock price
print(f"real return: {(v_real[v_real > 0][-1]/v_real[0] - 1)*100}%") # actual payoff / estimated price on the valuation date

real return: 6.119555449520564%


In [ ]:
SFD500.plot_real_mc_price(v_real, mc_daily_mean, mc_daily_median)

### Test a hedging strategy

In [37]:
ds = 0.001 * S0
hedge_result = SFD500.test_delta_hedge(S_real, v_real, r, q, sigma, n, ds)
hedge_result

,delta,bond,PnL,cumulative PnL
date,,,,
2019-01-03,30.542375,-59.508692,NaN,NaN
2019-01-04,32.451975,-72.679788,3.143270,3.143270
2019-01-07,31.117175,-66.736586,1.122341,4.265610
2019-01-08,26.979696,-50.216854,6.481927,10.747537
2019-01-09,27.663890,-53.739985,-0.276193,10.471345
2019-01-10,28.825104,-61.754077,1.485881,11.957225
2019-01-11,24.639134,-37.481904,-0.875123,11.082102
2019-01-14,28.955390,-56.341782,-4.323006,6.759096
2019-01-15,29.943012,-64.933801,3.147923,9.907020


In [74]:
SFD500.plot_delta_pnl(hedge_result["delta"], hedge_result["PnL"], hedge_result["cumulative PnL"], linewidth = 2)

In [53]:
print(f"Mean Delta: {hedge_result['delta'].mean()}\nMax Drardown: {structure_product.get_max_drawdown(hedge_result['cumulative PnL'])}%\nAnnual Sharpe: {structure_product.get_annual_sharpe(hedge_result['PnL'])}")

Mean Delta: 26.44218334515665
Max Drardown: 62.42184153773619%
Annual Sharpe: 2.953870450368852


In [47]:
hedge_result["delta"][hedge_result["delta"] > 0].describe()

count    21.000000
mean     27.701335
std       2.127614
min      24.184777
25%      26.337565
50%      27.663890
75%      28.825104
max      32.451975
Name: delta, dtype: float64

In [54]:
hedge_result["PnL"].describe()

count    21.000000
mean      0.591410
std       3.178322
min      -4.786452
25%      -1.022068
50%       1.037618
75%       3.133640
max       6.481927
Name: PnL, dtype: float64

In [49]:
hedge_result["cumulative PnL"]

date
2019-01-03          NaN
2019-01-04     3.143270
2019-01-07     4.265610
2019-01-08    10.747537
2019-01-09    10.471345
2019-01-10    11.957225
2019-01-11    11.082102
2019-01-14     6.759096
2019-01-15     9.907020
2019-01-16    14.455227
2019-01-17    14.081002
2019-01-18    15.937499
2019-01-21    19.071139
2019-01-22    15.089548
2019-01-23    12.319860
2019-01-24    13.357479
2019-01-25    16.124004
2019-01-28    15.477651
2019-01-29    12.975103
2019-01-30    11.953035
2019-01-31     7.166583
2019-02-01    12.419620
Name: cumulative PnL, dtype: float64